In [ ]:
!pip install catboost

In [2]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="150" alt="Gdrive-logo"/></center>
#@markdown <center><h2>Mount GDrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import string
import nltk
nltk.download('averaged_perceptron_tagger');

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.model_selection import StratifiedKFold, KFold
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams["figure.figsize"] = (12, 8)
pd.set_option('display.max_columns', None)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [25]:
# load the data
path = '/content/drive/MyDrive/COLAB/DATASETS/MAX_NG Data'
train = pd.read_csv(f'{path}/train.csv')
test = pd.read_csv(f'{path}/test.csv')
sub = pd.read_csv(f'{path}/sample_submission.csv')
random_seed = 2001 # random seed for all computations

In [26]:
# test.isna().sum()

In [27]:
# train[train.stage_status.isna() == False].head(20)

In [28]:
feat_1 = ['stage_status', 'agent_id']
feat_2 = ['subscriber_age', 'gender', 'city']
for data in (train, test):
    for feat in feat_1:
        data[feat] = data[feat].bfill().ffill()
    for feat_ in feat_2:
        data[feat_] = data[feat_].fillna(data[feat_].mode()[0])

In [29]:
# sns.set_style("white") 
# sns.countplot( y='label', data=train, palette="Set1")
# train[train.label == 'Backlkog']

In [30]:
# num_feats = list(train.select_dtypes(include=['int64', 'float64', 'int32']).columns)
# train[num_feats].hist(figsize=(20,15));

In [31]:
# test.subscriber_age > 90
# train = train.drop(train[train.subscriber_age > 90].index)

In [32]:
# sns.countplot(y = 'label', data = train, hue = "city", palette="Set1");

In [33]:
# train.info()

In [34]:
# train[train.city == 'ibadan']
train.city = train.city.str.replace('ibadan', 'Ibadan')

In [35]:
train = train.sort_values('date_time').reset_index(drop = True)
test = test.sort_values('date_time').reset_index(drop=True)

In [36]:
for data in (train, test):
    data['date_time'] = pd.to_datetime(data['date_time'])
    data['year'] = data.date_time.dt.year
    data['month'] = data.date_time.dt.month
    data['day'] = data.date_time.dt.day
    # data['hour'] = data.date_time.dt.hour
    # data['minute'] = data.date_time.dt.minute
    # data['second'] = data.date_time.dt.second
    data['year_month_day'] = data.year.astype(str) + '_' + data.month.astype(str)  + '_' + data.day.astype(str)
    data['month_day'] = data.month.astype(str)  + '_' + data.day.astype(str)
    data.drop('date_time', axis = 1, inplace=True)

In [37]:
ID = test['ID']
test.drop(['ID', 'gender'],inplace=True,axis=1)
train.drop(['ID', 'gender'],inplace=True,axis=1)

In [38]:
train.agent_id.nunique()

420

In [39]:
# df_mashup = [''.join(df[['subscriber_id', 'agent_id', 'stage_status']].astype(str).iloc[i].values) for i in range(len(df))]
# train_mashup = [''.join(train[['subscriber_id', 'agent_id', 'entry_channel_id', 'stage_status', 'city']].astype(str).iloc[i].values) for i in range(len(train))]
# test_mashup = [''.join(test[['subscriber_id', 'agent_id', 'entry_channel_id', 'stage_status', 'city']].astype(str).iloc[i].values) for i in range(len(test))]

In [40]:
# df = pd.concat([train, test])
# mashup = [''.join(df[['subscriber_id', 'agent_id', 'entry_channel_id', 'stage_status', 'city']].astype(str).iloc[i].values) for i in range(len(df))]
# mashup_cleaned = [mashup[i].strip().replace(',','') for i in range(len(mashup))]
# CouVec = CountVectorizer()
# CouVec.fit(df['agent_id'])

# train_words = pd.DataFrame(CouVec.transform(train['agent_id']).toarray())
# test_words = pd.DataFrame(CouVec.transform(test['agent_id']).toarray())

In [41]:
# train_words

In [42]:
# df[['subscriber_id', 'agent_id', 'entry_channel_id', 'stage_status', 'city']]

In [43]:
# LDA = LatentDirichletAllocation(n_components=15, max_iter=100, random_state=42)
# LDA.fit(pd.concat([train_words, test_words]))
# Topics = [f'Topic_{x}' for x in range(0,15)]
# train[Topics] = LDA.transform(train_words)
# test[Topics] = LDA.transform(test_words)

In [44]:
#Label Encoder
def label_enc(train_df, test_df, features):
    lbl_enc = LabelEncoder()
    full_data = pd.concat([train_df[features], test_df[features]],axis=0)
    for col in (features):
        print(col)
        lbl_enc.fit(full_data[col].values)
        train_df[col] = lbl_enc.transform(train_df[col])
        test_df[col] = lbl_enc.transform(test_df[col])
    return train_df, test_df

train, test = label_enc(train, test, ['subscriber_id', 'agent_id', 'entry_channel_id', 'stage_status', 'city', 'month_day', 'year_month_day'])

subscriber_id
agent_id
entry_channel_id
stage_status
city
month_day
year_month_day


In [47]:
# train.info()
train.shape, test.shape

((214847, 15), (92078, 14))

In [50]:
# #Duplicated
# cols = train.columns
# dup = []
# for feat_1 in cols:
#     if (feat_1 in dup):
#         continue
#     for feat_2 in cols.drop(feat_1):
#         if (feat_2 in dup):
#             continue
#         if (train[feat_1].equals(train[feat_2])):
#             train.drop(feat_2,inplace=True,axis=1)
#             test.drop(feat_2,inplace=True,axis=1)
#             dup.append(feat_2)

In [ ]:
from sklearn.utils import resample
#create two different dataframe of majority and minority class
df_majority = train[(train['label'] != 'Backlog')]
df_minority = train[(train['label'] == 'Backlog')]
# upsample minority class
df_minority_upsampled = resample(df_minority,
                                 replace=True,    # sample with replacement
                                 n_samples= 88, # to match majority class
                                 random_state=42)  # reproducible results
# Combine majority class with upsampled minority class
df = pd.concat([df_minority_upsampled, df_majority])
df['label'].value_counts(normalize=True).plot.bar()
df['label'].value_counts()
df.shape, train.shape

In [68]:
# df.label.value_counts()

In [69]:
X, y = df.drop('label', axis = 1), df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .15, shuffle = True, random_state = random_seed)

In [70]:
d_train, d_test  = Pool(X_train, y_train), Pool(X_test, y_test)
cb_model_ = CatBoostClassifier(l2_leaf_reg = 9.441413522475084, depth = 7, bootstrap_type = 'Bayesian', learning_rate = 0.01772339213540557, n_estimators = 3167, use_best_model = True,
                                                 leaf_estimation_iterations = 1, random_strength = 0.17095032711212016, loss_function = 'MultiClass', verbose = 0, random_state = random_seed)
cb_model_.fit(d_train, eval_set = [(d_test)], verbose = 0, early_stopping_rounds = 500)
preds_ = cb_model_.predict_proba(d_test)
log_loss(y_test, preds_).round(5)

1.74341

In [75]:
lgb_model_ = LGBMClassifier(boosting_type = 'gbdt', objective = 'multiclass', metric = 'multi_logloss', n_estimators = 3000, learning_rate = 0.01, use_best_model = True,
                                             num_leaves = 45, colsample_bytree = 0.5, subsample = 0.9, subsample_freq = 1, max_depth = 6, reg_alpha = 0.8, reg_lambda = 0.8,
                                             min_split_gain = 0.05, min_child_weight = 0.05, random_state = random_seed, num_class = 12, silent = -1, verbose = -1)
lgb_model_.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], early_stopping_rounds = 500, eval_metric = 'logloss', verbose = 0)
preds_ = lgb_model_.predict_proba(X_test)
log_loss(y_test, preds_).round(5)

1.73614

In [ ]:
# creating fols to n=be used for cross validation
TARGET_COL = 'label'
remove_features = ['label', 'folds']
features_columns = [col for col in df.columns if col not in remove_features]
cat = le.fit_transform(df.label)
def create_folds(data):
    data["folds"] = -1
    data = data.sample(frac = 1).reset_index(drop = True)
    num_bins = np.floor(1 + np.log2(len(train))).astype(int)
    data.loc[:, "bins"] = pd.cut(cat, bins = num_bins, labels = False)
    kf = StratifiedKFold(n_splits = 15)
    for f, (t_, v_) in enumerate(kf.split(X = data, y = data.bins.values)):
        data.loc[v_, "folds"] = f
    data.drop("bins", axis = 1, inplace = True)
    return data
train = create_folds(df)

In [ ]:
log_loss_score_ = []
print("-" * 30)
n_splits = 15
for fold in range(n_splits):
  x_train_, y_train_ = train[train['folds']!=fold][features_columns] , train[train['folds']!=fold][TARGET_COL] 
  x_test_, y_test_ = train[train['folds']==fold][features_columns] , train[train['folds']==fold][TARGET_COL] 
  d_train = Pool(x_train_, y_train_)
  d_test = Pool(x_test_, y_test_)
  model_cb = CatBoostClassifier(l2_leaf_reg = 9.441413522475084, depth = 7, bootstrap_type = 'Bayesian', learning_rate = 0.01772339213540557, n_estimators = 3167, use_best_model = True,
                                                 leaf_estimation_iterations = 1, random_strength = 0.17095032711212016, loss_function = 'MultiClass', verbose = 0, random_state = random_seed)
  model_cb.fit(d_train, eval_set = [(d_train), (d_test)], verbose = 0, early_stopping_rounds = 500)
  preds_ = model_cb.predict_proba(d_test)
  log_loss_ = log_loss(y_test_, preds_)
  print(f'LOG_LOSS_{fold + 1}: {log_loss_}')
  log_loss_score_.append(log_loss_)
  print("-" * 30)
print('\n')
print(f"LOG_LOSS_CV_CB: {np.mean(log_loss_score_).round(5)}")

In [ ]:
log_loss_score_lgb = []
print("-" * 30)
n_splits = 15
for fold in range(n_splits):
  x_train_, y_train_ = train[train['folds']!=fold][features_columns] , train[train['folds']!=fold][TARGET_COL] 
  x_test_, y_test_ = train[train['folds']==fold][features_columns] , train[train['folds']==fold][TARGET_COL] 
  model_lgb = LGBMClassifier(boosting_type = 'gbdt', objective = 'multiclass', metric = 'multi_logloss', n_estimators = 3000, learning_rate = 0.01, 
                              num_leaves = 45, colsample_bytree = 0.8, subsample = 0.9, subsample_freq = 1, max_depth = 6, reg_alpha = 0.5, reg_lambda = 0.5, 
                              min_split_gain = 0.05, min_child_weight = 0.05, random_state = random_seed, num_class = 13, silent = -1, verbose = -1)
  model_lgb.fit(x_train_, y_train_, eval_set = [(x_train_, y_train_), (x_test_, y_test_)], early_stopping_rounds = 300, eval_metric = 'logloss', verbose = 0)
  preds_ = model_lgb.predict_proba(x_test_)
  log_loss_ = log_loss(y_test_, preds_)
  print(f'LOG_LOSS_{fold + 1}: {log_loss_}')
  log_loss_score_lgb.append(log_loss_)
  print("-" * 30)
print('\n')
print(f"LOG_LOSS_CV_LGB: {np.mean(log_loss_score_lgb).round(5)}")

In [72]:
def predict_and_submit(test_, filename):
    d_ = {'ID' : ID, 'Activated' : test_[:, 0], 'Awaiting Activation' : test_[:, 1], 'Awaiting Vehicle Pickup' : test_[:, 2], 'Backlog' : test_[:, 3], 'Checked In For Test' : test_[:, 4], 'In Verification' : test_[:, 5],
         'Issued Guarantor Form' : test_[:, 6], 'Onboarding' : test_[:, 7], 'Received Guarantor Form' : test_[:, 8], 'Test Scheduled' : test_[:, 9], 'Tested' : test_[:, 10], 'Top of the funnel' : test_[:, 11]}
    df_ = pd.DataFrame(data = d_)
    df_ = df_[['ID', 'Activated', 'Awaiting Activation', 'Awaiting Vehicle Pickup',
       'Backlog', 'Checked In For Test', 'In Verification',
       'Issued Guarantor Form', 'Onboarding', 'Received Guarantor Form',
       'Test Scheduled', 'Tested', 'Top of the funnel']]
    df_.to_csv(f'{filename}.csv', index = False)
    return df_.shape

In [ ]:
y_a = cb_model_.predict_proba(test)
y_b = lgb_model_.predict_proba(test)
# y_c = model_cb.predict_proba(test)
# y_d = model_lgb.predict_proba(test)

In [ ]:
# pred = 0.5 * y_b + 0.5 * y_b
pred = y_b+
predict_and_submit(pred, 'ai_cat_1.1_')